# Day 09. Exercise 04
# Pipelines and OOP

## 0. Imports

In [7]:
import pandas as pd
import numpy as np
import joblib
from tqdm.notebook import tqdm

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

## 1. Preprocessing pipeline

Создайте три пользовательских трансформатора, первые два из которых будут использоваться в [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

1. Класс `FeatureExtractor()`:
 - Берет датафрейм с `uid`, `labname`, `numTrials`, `timestamp` из файла [`checker_submits.csv`](https://drive.google.com/file/d/14voc4fNJZiLEFaZyd8nEG-lQt5JjatYw/view?usp=sharing).
 - Извлекает `час` из `временной метки`.
 - Извлекает `weekday` из `timestamp` (числа).
 - Опускает столбец `timestamp`.
 - Возвращает новый фрейм данных.


2. Класс `MyOneHotEncoder()`:
 - Берет датафрейм из результата предыдущего преобразования и имя целевого столбца.
 - Определяет все категориальные признаки и преобразует их с помощью `OneHotEncoder()`. Если целевой столбец тоже категориальный, то преобразование к нему не применяется.
 - Отбрасывает исходные категориальные признаки.
 - Возвращает кадр данных с признаками и серию с целевым столбцом.


3. Класс `TrainValidationTest()`:
 - Берет `x` и `y`.
 - Возвращает `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` (`test_size=0.2`, `random_state=21`, `stratified`).

In [9]:
class FeatureExtractor():
    def __init__(self):
        pass

    def fit(self, x=None, y=None):
        return self

    def transform(self, x, y=None):
        x['hour'] = x['timestamp'].dt.hour
        x['dayofweek'] = x['timestamp'].dt.weekday
        x = x.drop(['timestamp'], axis=1)
        return x

In [10]:
class MyOneHotEncoder():
    def __init__(self, target):
        self.categorical_columns = None
        self.target = target
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    def fit(self, x, y=None):
        self.categorical_columns = x.select_dtypes(include=['object', 'category']).columns.tolist()
        if self.target in self.categorical_columns:
            self.categorical_columns.remove(self.target)
        self.encoder.fit(x[self.categorical_columns])

        return self

    def transform(self, x, y=None):
        encod_trans = self.encoder.transform(x[self.categorical_columns])
        encod_df = pd.DataFrame(encod_trans, columns=self.encoder.get_feature_names_out(self.categorical_columns))
        x = x.drop(columns=self.categorical_columns)
        x = pd.concat([x, encod_df], axis=1)
        return x

In [11]:
class TrainValidationTest():
    def __init__(self):
        self.test_size = 0.2
        self.random_state = 21

    def split(self, x, y):
        x_val, x_test, y_val, y_test = train_test_split(x, y, test_size=self.test_size, random_state=self.random_state, stratify=y)
        x_train, x_valid, y_train, y_valid = train_test_split(x_val, y_val, test_size=self.test_size, random_state=self.random_state, stratify=y_val)

        return x_train, x_valid, x_test, y_train, y_valid, y_test

In [12]:
# fe = FeatureExtractor()
# df = fe.read_file()

# mohe = MyOneHotEncoder()
# df = mohe.fit_transform(df, 'dayofweek')

# tvt = TrainValidationTest()
# x = df.drop(['dayofweek'], axis=1)
# y = df['dayofweek']
# x_train, x_valid, x_test, y_train, y_valid, y_test = tvt.split(x, y)

## 2. Model selection pipeline

`ModelSelection()` class

 - Принимает список экземпляров `GridSearchCV` и dict, где ключами являются индексы из этого списка, а значениями - имена моделей, пример приведен ниже в обратном порядке (с высокого уровня к низкому):

```
ModelSelection(grids, grid_dict)

grids = [gs_svm, gs_tree, gs_rf]

gs_svm = GridSearchCV(estimator=svm, param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=jobs), where jobs you can specify by yourself

svm_params = [{'kernel':('linear', 'rbf', 'sigmoid'), 'C':[0.01, 0.1, 1, 1.5, 5, 10], 'gamma': ['scale', 'auto'], 'class_weight':('balanced', None), 'random_state':[21], 'probability':[True]}]
```

 - Метод `choose()` берет `X_train`, `y_train`, `X_valid`, `y_valid` и возвращает имя лучшего классификатора среди всех моделей на валидном множестве
 - Метод `best_results()` возвращает фрейм данных со столбцами `model`, `params`, `valid_score`, где строки - это лучшие модели в каждом классе моделей.

```
model	params	valid_score
0	SVM	{'C': 10, 'class_weight': None, 'gamma': 'auto...	0.772727
1	Decision Tree	{'class_weight': 'balanced', 'criterion': 'gin...	0.801484
2	Random Forest	{'class_weight': None, 'criterion': 'entropy',...	0.855288
```

 - Когда вы перебираете параметры класса модели, выводите имя этого класса и показываете прогресс с помощью `tqdm.notebook`, в конце цикла выведите лучшую модель этого класса.

```
Estimator: SVM
100%
125/125 [01:32<00:00, 1.36it/s]
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878 

Estimator: Decision Tree
100%
57/57 [01:07<00:00, 1.22it/s]
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 21, 'random_state': 21}
Best training accuracy: 0.801
Validation set accuracy score for best params: 0.867 

Estimator: Random Forest
100%
284/284 [06:47<00:00, 1.13s/it]
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907 

Classifier with best validation set accuracy: Random Forest
```

In [13]:
svm_params = [{'kernel': ['linear', 'rbf', 'sigmoid'],
              'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None],
              'random_state': [21],
              'probability': [True]}]

dtc_params = [{'max_depth': np.arange(1, 30, 1),
               'class_weight': ['balanced', None],
               'criterion': ['entropy', 'gini'],
               'random_state': [21]}]

rfc_params = [{'n_estimators': [5, 10, 50, 100],
              'max_depth': np.arange(1, 49, 1),
              'class_weight': ['balanced', None],
              'criterion': ['entropy', 'gini'],
              'random_state': [21]}]

gs_svm = GridSearchCV(estimator=SVC(), param_grid=svm_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_tree = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=dtc_params, scoring='accuracy', cv=2, n_jobs=-1)
gs_rf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=rfc_params, scoring='accuracy', cv=2, n_jobs=-1)

grids = [gs_svm, gs_tree, gs_rf]
grid_dict = {0: 'SVM', 1: 'Decision Tree', 2: 'Random Forest'}

In [14]:
class ModelSelection:
    def __init__(self, grids, grid_dict):
        self.grids = grids
        self.grid_dict = grid_dict
        self.results = []

    def choose(self, X_train, y_train, X_valid, y_valid):
        best_model_name = None
        best_valid_score = 0

        for i, grid in enumerate(self.grids):
            model_name = self.grid_dict[i]
            print(f'Estimator: {model_name}')

            grid.fit(X_train, y_train)
            y_valid_pred = grid.predict(X_valid)
            valid_score = accuracy_score(y_valid, y_valid_pred)

            self.results.append({
                'model': model_name,
                'params': grid.best_params_,
                'valid_score': valid_score
            })

            print(f'Best params: {grid.best_params_}')
            print(f'Best training accuracy: {grid.best_score_:.3f}')
            print(f'Validation set accuracy score for best params: {valid_score:.3f}\n')
            if valid_score > best_valid_score:
                best_valid_score = valid_score
                best_model_name = model_name
        print(f'Classifier with best validation set accuracy: {best_model_name}')
        return best_model_name

    def best_results(self):
        return pd.DataFrame(self.results)

In [15]:
# model_selector = ModelSelection(grids, grid_dict)
# best_model_name = model_selector.choose(x_train, y_train, x_valid, y_valid)
# result = model_selector.best_results()
# result

## 3. Finalization

`Finalize()` class
 - Принимает оценщик.
 - Метод `final_score()` принимает `X_train`, `y_train`, `X_test`, `y_test` и возвращает точность модели, как в примере ниже:
```
final.final_score(X_train, y_train, X_test, y_test)
Accuracy of the final model is 0.908284023668639
```
 - Метод `save_model()` принимает путь, сохраняет модель по этому пути и печатает, что модель была успешно сохранена.

In [16]:
class Finalize:
    def __init__(self, estimator):
        self.estimator = estimator

    def final_score(self, x_train, y_train, X_test, y_test):
        self.estimator.fit(x_train, y_train)
        y_test_pred = self.estimator.predict(X_test)

        test_accuracy = accuracy_score(y_test, y_test_pred)
        print(f'Accuracy of the final model is {test_accuracy:.6f}')
        return test_accuracy

    def save_model(self, path):
        joblib.dump(self.estimator, path)
        print(f'Model successfully saved to {path}')

In [17]:
# model = RandomForestClassifier(random_state=21)
# final = Finalize(model)
# final.final_score(x_train, y_train, x_test, y_test)
# final.save_model('./final_model.pkl')

## 4. Main program

1. Загрузите данные из файла (**** имя файла****).
2. Создайте конвейер предварительной обработки, состоящий из двух пользовательских трансформаторов: `FeatureExtractor()` и `MyOneHotEncoder()`:
```
preprocessing = Pipeline([('feature_extractor', FeatureExtractor()), ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
```
3. Используйте этот конвейер и его метод `fit_transform()` на исходном наборе данных.
```
data = preprocessing.fit_transform(df)
```
4. Получите `X_train`, `X_valid`, `X_test`, `y_train`, `y_valid`, `y_test` с помощью `TrainValidationTest()` и результат конвейера.
5. Создайте экземпляр `ModelSelection()`, используйте метод `choose()`, применяя его к нужным вам моделям и параметрам, получите датафрейм лучших результатов.
6. создайте экземпляр `Finalize()` с вашей лучшей моделью, используйте метод `final_score()` и сохраните модель в формате: `имя_модели_{точность на тестовом наборе данных}.sav`.

Вот и все, поздравляем!

In [18]:
df = pd.read_csv('../data/checker_submits.csv', parse_dates=['timestamp'])

preprocessing = Pipeline([('feature_extractor', FeatureExtractor()),
                          ('onehot_encoder', MyOneHotEncoder('dayofweek'))])
data = preprocessing.fit_transform(df)

x = data.drop(['dayofweek'], axis=1)
y = data['dayofweek']

tvt = TrainValidationTest()
x_train, x_valid, x_test, y_train, y_valid, y_test = tvt.split(x, y)

model_selector = ModelSelection(grids, grid_dict)
best_model_name = model_selector.choose(x_train, y_train, x_valid, y_valid)
result = model_selector.best_results()
print(result)

best_model = None
for i, grid in enumerate(grids):
    if grid_dict[i] == best_model_name:
        best_model = grid.best_estimator_
        break

final = Finalize(best_model)
test_accuracy = final.final_score(x_train, y_train, x_test, y_test)

model_file_name = f"{best_model_name}_{test_accuracy:.5f}.sav"
final.save_model(model_file_name)

Estimator: SVM
Best params: {'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf', 'probability': True, 'random_state': 21}
Best training accuracy: 0.773
Validation set accuracy score for best params: 0.878

Estimator: Decision Tree
Best params: {'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': np.int64(22), 'random_state': 21}
Best training accuracy: 0.809
Validation set accuracy score for best params: 0.867

Estimator: Random Forest
Best params: {'class_weight': None, 'criterion': 'entropy', 'max_depth': np.int64(22), 'n_estimators': 50, 'random_state': 21}
Best training accuracy: 0.855
Validation set accuracy score for best params: 0.907

Classifier with best validation set accuracy: Random Forest
           model                                             params  \
0            SVM  {'C': 10, 'class_weight': None, 'gamma': 'auto...   
1  Decision Tree  {'class_weight': 'balanced', 'criterion': 'gin...   
2  Random Forest  {'class_weight': None, 'criterion':